In [7]:

import os
from copy import deepcopy
# import psutil
import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt

In [8]:
clean_images_dir = 'data/upscaled'
watermark_images_dir = 'data/watermark_upscaled'
segmented_watermarks_dir = 'data/segmented'


In [9]:

def sort_by_index(_strs):
    import re
    def str_index(_str):
        match = re.search(r'_(\d+)', _str)
        return int(match.group(1)) if match else float('inf')
    
    return sorted(_strs, key=str_index)


def get_image_paths(dir, n=1):
    valid_ext = ['.jpg', '.jpeg', '.png', '.gif', '.bmp']
    paths = [os.path.join(dir, filename) for filename in sort_by_index(os.listdir(dir))[:n]
             if any(filename.lower().endswith(ext) for ext in valid_ext)]
    return paths

test_clean_images_path = get_image_paths(clean_images_dir, n=10)
test_watermark_images_path = get_image_paths(watermark_images_dir, n=10)

print(test_clean_images_path, test_watermark_images_path, sep='\n')

['data/upscaled\\image_0.jpg', 'data/upscaled\\image_1.jpg', 'data/upscaled\\image_2.jpg', 'data/upscaled\\image_3.jpg', 'data/upscaled\\image_4.jpg', 'data/upscaled\\image_5.jpg', 'data/upscaled\\image_6.jpg', 'data/upscaled\\image_7.jpg', 'data/upscaled\\image_8.jpg', 'data/upscaled\\image_9.jpg']
['data/watermark_upscaled\\image_0.png', 'data/watermark_upscaled\\image_1.png', 'data/watermark_upscaled\\image_2.png', 'data/watermark_upscaled\\image_3.png', 'data/watermark_upscaled\\image_4.png', 'data/watermark_upscaled\\image_5.png', 'data/watermark_upscaled\\image_6.png', 'data/watermark_upscaled\\image_7.png', 'data/watermark_upscaled\\image_8.png', 'data/watermark_upscaled\\image_9.jpg']


In [ ]:
def plot_images_grid(images, rows, cols):
    
    
    return

In [ ]:
def segment_watermark(clean_path, watermark_path):
    clean_gray = cv2.imread(clean_path, cv2.IMREAD_GRAYSCALE)
    watermark_gray = cv2.imread(watermark_path, cv2.IMREAD_GRAYSCALE)
    
    if clean_gray is None or watermark_gray is None:
        raise ValueError("One or both of the images could not be loaded.")

    diff = cv2.absdiff(clean_gray, watermark_gray)
    _, thresh_segmented = cv2.threshold(diff, 20, 255, cv2.THRESH_BINARY)
    return thresh_segmented


test_segmented = [ segment_watermark(clean_path, wm_path)  
                  for clean_path, wm_path in zip(test_clean_images_path , test_watermark_images_path)]




# cv2.imshow("Window Name", test_segmented[0])
# cv2.waitKey(0)
# cv2.destroyAllWindows()


In [11]:
class WatermarkDataset(torch.utils.data.Dataset):
    """Some Information about WatermarkDataset"""
    def __init__(self):
        super(WatermarkDataset, self).__init__()

    def __getitem__(self, index):
        return 

    def __len__(self):
        return 